In [ ]:
import cv2
import mediapipe as mp
import time
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module='google.protobuf.symbol_database')

estado = False
modo = 1

# Inicializar MediaPipe 
mpHands = mp.solutions.hands
# Configuración de MediaPipe para detectar manos
hands = mpHands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7, max_num_hands=2)

mpDraw = mp.solutions.drawing_utils

# Variables para el modo 1
operacion_actual = ""
resultado = ""
ultimo_numero = ""
tiempo_ultima_deteccion = time.time()
intervalo_deteccion = 3
tiempo_reinicio = 5
mostrar_resultado = False


chuleta_imagenes = [
    cv2.imread('SUMA.jpg'),
    cv2.imread('RESTA.jpg'),
    cv2.imread('MULTIPLICACION.jpg'),
    cv2.imread('DIVISION.jpg'),
    cv2.imread('IGUAL.jpg')
]
salir_imagen = cv2.imread('SALIR.png')

chuleta_imagenes = [cv2.resize(img, (100, 100)) for img in chuleta_imagenes]
salir_imagen = cv2.resize(salir_imagen, (100, 100))



2025-01-08 16:16:13.442526: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1736352986.563668 6639404 gl_context.cc:357] GL version: 2.1 (2.1 INTEL-18.7.4), renderer: Intel(R) Iris(TM) Plus Graphics 640
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1736352986.679653 6640337 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1736352986.755394 6640336 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [4]:
# Función modo 1 para detectar gestos de manos 
def modo_1(frame):
    cv2.putText(frame, f"CALCULADORA MODO 1", (400, 40), cv2.FONT_HERSHEY_COMPLEX, 1.5, (255, 69, 0), 3, lineType=cv2.LINE_AA)
    cv2.putText(frame, f"(Pulsa la tecla '2' para el modo 2)", (490, 80), cv2.FONT_HERSHEY_SIMPLEX , 0.6, (255, 0, 0), 1, lineType=cv2.LINE_AA)

    global operacion_actual, resultado, ultimo_numero, tiempo_ultima_deteccion, mostrar_resultado

    imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)

    if results.multi_hand_landmarks and not mostrar_resultado:
        for handLms in results.multi_hand_landmarks:
            # Coordenadas de los puntos relevantes de la mano
            landmarks = handLms.landmark

            dedos_levantados = {
                "pulgar": landmarks[4].x > landmarks[3].x,
                "indice": landmarks[8].y < landmarks[6].y,
                "medio": landmarks[12].y < landmarks[10].y,
                "anular": landmarks[16].y < landmarks[14].y,
                "meñique": landmarks[20].y < landmarks[18].y,
            }

            # Contar los dedos levantados que no sean el meñique ni el pulgar ya que son signos
            dedos_detectados = sum([
                dedos_levantados["indice"],
                dedos_levantados["medio"],
                dedos_levantados["anular"]
            ]) + dedos_levantados["pulgar"] + dedos_levantados["meñique"]

            tiempo_actual = time.time()

            # RESTAR
            if (
                dedos_levantados["meñique"]
                and not dedos_levantados["indice"]
                and not dedos_levantados["medio"]
                and not dedos_levantados["anular"]
                and not dedos_levantados["pulgar"]
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                operacion_actual += "-"
                resultado += ultimo_numero + "-"
                ultimo_numero = ""

            #SUMAR
            elif (
                dedos_levantados["pulgar"]
                and dedos_levantados["meñique"]
                and not dedos_levantados["indice"]
                and not dedos_levantados["medio"]
                and not dedos_levantados["anular"]
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                operacion_actual += "+"
                resultado += ultimo_numero + "+"
                ultimo_numero = ""
            
            #MULTIPLICAR
            elif (
                dedos_levantados["pulgar"]
                and dedos_levantados["meñique"]
                and dedos_levantados["indice"]
                and not dedos_levantados["medio"]
                and not dedos_levantados["anular"]
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                operacion_actual += "*"
                resultado += ultimo_numero + "*"
                ultimo_numero = ""
            
            #DIVIDIR
            elif (
                dedos_levantados["pulgar"]
                and dedos_levantados["meñique"]
                and dedos_levantados["indice"]
                and dedos_levantados["medio"]
                and not dedos_levantados["anular"]
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                operacion_actual += "/"
                resultado += ultimo_numero + "/"
                ultimo_numero = ""

            #IGUAL
            elif (
                dedos_levantados["indice"]
                and dedos_levantados["meñique"]
                and not dedos_levantados["medio"]
                and not dedos_levantados["anular"]
                and not dedos_levantados["pulgar"]
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                resultado += ultimo_numero
                operacion_actual += "="

                try:
                    calculo = eval(resultado)
                    resultado += f"={calculo}"
                except:
                    resultado += "=Error"
                ultimo_numero = ""
                operacion_actual = ""
                mostrar_resultado = True  # Bloquear después de "="

            elif (
                dedos_detectados >= 0
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                if dedos_detectados == 0:
                    ultimo_numero += "0"
                elif dedos_detectados == 1:
                    ultimo_numero += "1"
                elif dedos_detectados == 2:
                    ultimo_numero += "2"
                elif dedos_detectados == 3:
                    ultimo_numero += "3"
                elif dedos_detectados == 4:
                    ultimo_numero += "4"
                elif dedos_detectados == 5:
                    ultimo_numero += "5"
                
                

            # Dibujar las conexiones de la mano
            mpDraw.draw_landmarks(frame, handLms, mpHands.HAND_CONNECTIONS)

    # Si han pasado 5 segundos desde la última detección, reinicia la operación
    if time.time() - tiempo_ultima_deteccion > tiempo_reinicio:
        operacion_actual = ""
        resultado = ""
        ultimo_numero = ""
        mostrar_resultado = False  # Desbloquear después del reinicio
    
    alto_ventana, ancho_ventana, _ = frame.shape

    espacio_imagen = 110  
    x_offset = ancho_ventana - (len(chuleta_imagenes) * espacio_imagen) - 5  
    y_offset = alto_ventana - 110  

    for i, imagen in enumerate(chuleta_imagenes):
        x_pos = x_offset + i * espacio_imagen
        frame[y_offset:y_offset + 100, x_pos:x_pos + 100] = imagen

    # Resultado en pantalla
    cv2.putText(frame, f"OPERACION: ", (20, 180), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"{resultado}{ultimo_numero}", (20, 280), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 255, 0), 6)



In [3]:
import random

# Función para el modo 2
def modo_2(frame):
    global operacion_actual, resultado_usuario, tiempo_ultima_deteccion, mostrar_resultado, resultado_correcto, operacion_generada

    cv2.putText(frame, f"CALCULADORA MODO 2", (400, 40), cv2.FONT_HERSHEY_COMPLEX, 1.5, (255, 69, 0), 3, lineType=cv2.LINE_AA)
    cv2.putText(frame, f"(Pulsa la tecla '1' para el modo 1)", (490, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, lineType=cv2.LINE_AA)

    # Generar operación aleatoria si no existe ninguna
    if not operacion_actual:
        while True:
            num1 = random.randint(1, 5) * 10 + random.randint(1, 5) 
            num2 = random.randint(1, 5) * 10 + random.randint(1, 5)
            operador = random.choice(['+', '-', '*', '/'])

            # Calcular el resultado
            if operador == '+':
                resultado_correcto = num1 + num2
            elif operador == '-':
                resultado_correcto = num1 - num2
            elif operador == '*':
                resultado_correcto = num1 * num2
            elif operador == '/':
                if num2 != 0 and num1 % num2 == 0:  # Asegurar divisiones enteras
                    resultado_correcto = num1 // num2
                else:
                    continue  # Volver a intentar si la división no es válida

            
            resultado_str = str(resultado_correcto)
            if resultado_correcto > 0 and all(1 <= int(d) <= 5 for d in resultado_str):
                operacion_actual = f"{num1}{operador}{num2}"
                break

        resultado_usuario = ""
        mostrar_resultado = False

    imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)

    if results.multi_hand_landmarks and not mostrar_resultado:
        for handLms in results.multi_hand_landmarks:
            landmarks = handLms.landmark

            dedos_levantados = {
                "pulgar": landmarks[4].x > landmarks[3].x,
                "indice": landmarks[8].y < landmarks[6].y,
                "medio": landmarks[12].y < landmarks[10].y,
                "anular": landmarks[16].y < landmarks[14].y,
                "meñique": landmarks[20].y < landmarks[18].y,
            }

            # Contar dedos 
            dedos_detectados = sum(dedos_levantados.values())

            tiempo_actual = time.time()

            # Detectar gesto de igual para validar resultado
            if (
                dedos_levantados["indice"]
                and dedos_levantados["meñique"]
                and not dedos_levantados["medio"]
                and not dedos_levantados["anular"]
                and not dedos_levantados["pulgar"]
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                mostrar_resultado = True

            elif dedos_detectados >= 0 and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion:
                tiempo_ultima_deteccion = tiempo_actual
                if dedos_detectados == 0:
                    resultado_usuario += "0"
                elif dedos_detectados == 1:
                    resultado_usuario += "1"
                elif dedos_detectados == 2:
                    resultado_usuario += "2"
                elif dedos_detectados == 3:
                    resultado_usuario += "3"
                elif dedos_detectados == 4:
                    resultado_usuario += "4"
                elif dedos_detectados == 5:
                    resultado_usuario += "5"

    
            mpDraw.draw_landmarks(frame, handLms, mpHands.HAND_CONNECTIONS)

    # Mostrar la operación y el resultado ingresado
    cv2.putText(frame, f"Operacion: {operacion_actual}", (20, 180), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Respuesta: {resultado_usuario}", (20, 240), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    # Validar si el resultado es correcto o incorrecto y mostrarlo en pantalla
    if mostrar_resultado:
        if int(resultado_usuario) == resultado_correcto:
            cv2.putText(frame, f"CORRECTO!", (20, 300), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)
        else:
            cv2.putText(frame, f"INCORRECTO!", (20, 300), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)
            cv2.putText(frame, f"Correcto: {resultado_correcto}", (20, 360), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

        # Reiniciar después de mostrar el resultado 
        if time.time() - tiempo_ultima_deteccion > tiempo_reinicio:
            operacion_actual = ""
            resultado_usuario = ""
            mostrar_resultado = False

In [15]:
# Inicializar cámara
import os

cam = cv2.VideoCapture(0)

# Configuración para guardar el video de salida
output_video_path = os.path.join(os.getcwd(), 'output_resultado.mp4')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Establecer FPS y dimensiones válidas
fps = cam.get(cv2.CAP_PROP_FPS)
if fps == 0:  # Si FPS es 0, asignar un valor por defecto
    fps = 30.0

frame_width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Crear el objeto VideoWriter
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

while True:
    ret, frame = cam.read()
    if not ret:
        break

    key = cv2.waitKey(1) & 0xFF
    if key == ord('1'):  # cambiar a modo 1 con la tecla '1'
        modo = 1
    if key == ord('2'):  # cambiar a modo 2 con la tecla '2'
        modo = 2

    if modo == 1:  
        modo_1(frame)
    elif modo == 2:  
        modo_2(frame)

    cv2.imshow("Modo Interactivo", frame)

    if key == ord('q'):
        break

cam.release()
out.release()
cv2.destroyAllWindows()

In [61]:
import cv2
import mediapipe as mp
import time
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module='google.protobuf.symbol_database')

# Inicializar MediaPipe
mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

# Captura de video
cam = cv2.VideoCapture(0)

# Variables para mantener el estado
operacion_actual = ""
resultado = ""
ultimo_numero = ""
tiempo_ultima_deteccion = time.time()
intervalo_deteccion = 3 
tiempo_reinicio = 5  # Tiempo de reinicio cuando hay espera
mostrar_resultado = False  # Bandera para bloquear la entrada después del "="


chuleta_imagenes = [
    cv2.imread('SUMA.jpg'),
    cv2.imread('RESTA.jpg'),
    cv2.imread('MULTIPLICACION.jpg'),
    cv2.imread('DIVISION.jpg'),
    cv2.imread('IGUAL.jpg')
]

chuleta_imagenes = [cv2.resize(img, (100, 100)) for img in chuleta_imagenes]

while True:
    success, img = cam.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)

    if results.multi_hand_landmarks and not mostrar_resultado:
        for handLms in results.multi_hand_landmarks:
            # Coordenadas de los puntos relevantes de la mano
            landmarks = handLms.landmark

            # Determinar si los dedos están levantados
            dedos_levantados = {
                "pulgar": landmarks[4].x > landmarks[3].x,
                "indice": landmarks[8].y < landmarks[6].y,
                "medio": landmarks[12].y < landmarks[10].y,
                "anular": landmarks[16].y < landmarks[14].y,
                "meñique": landmarks[20].y < landmarks[18].y,
            }

            # Contar los dedos levantados que no sean el meñique ni el pulgar
            dedos_detectados = sum([
                dedos_levantados["indice"],
                dedos_levantados["medio"],
                dedos_levantados["anular"]
            ]) + dedos_levantados["pulgar"] + dedos_levantados["meñique"]

            # Tiempo actual para el temporizador
            tiempo_actual = time.time()

            # RESTAR
            if (
                dedos_levantados["meñique"]
                and not dedos_levantados["indice"]
                and not dedos_levantados["medio"]
                and not dedos_levantados["anular"]
                and not dedos_levantados["pulgar"]
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                operacion_actual += "-"
                resultado += ultimo_numero + "-"
                ultimo_numero = ""

            #SUMAR
            elif (
                dedos_levantados["pulgar"]
                and dedos_levantados["meñique"]
                and not dedos_levantados["indice"]
                and not dedos_levantados["medio"]
                and not dedos_levantados["anular"]
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                operacion_actual += "+"
                resultado += ultimo_numero + "+"
                ultimo_numero = ""
            
            #MULTIPLICAR
            elif (
                dedos_levantados["pulgar"]
                and dedos_levantados["meñique"]
                and dedos_levantados["indice"]
                and not dedos_levantados["medio"]
                and not dedos_levantados["anular"]
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                operacion_actual += "*"
                resultado += ultimo_numero + "*"
                ultimo_numero = ""
            
            #DIVIDIR
            elif (
                dedos_levantados["pulgar"]
                and dedos_levantados["meñique"]
                and dedos_levantados["indice"]
                and dedos_levantados["medio"]
                and not dedos_levantados["anular"]
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                operacion_actual += "/"
                resultado += ultimo_numero + "/"
                ultimo_numero = ""

            #IGUAL
            elif (
                dedos_levantados["indice"]
                and dedos_levantados["meñique"]
                and not dedos_levantados["medio"]
                and not dedos_levantados["anular"]
                and not dedos_levantados["pulgar"]
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                resultado += ultimo_numero
                operacion_actual += "="

                try:
                    calculo = eval(resultado)
                    resultado += f"={calculo}"
                except:
                    resultado += "=Error"
                ultimo_numero = ""
                operacion_actual = ""
                mostrar_resultado = True  # Bloquear después de "="

            elif (
                dedos_detectados >= 0
                and tiempo_actual - tiempo_ultima_deteccion > intervalo_deteccion
            ):
                tiempo_ultima_deteccion = tiempo_actual
                if dedos_detectados == 0:
                    ultimo_numero += "0"
                elif dedos_detectados == 1:
                    ultimo_numero += "1"
                elif dedos_detectados == 2:
                    ultimo_numero += "2"
                elif dedos_detectados == 3:
                    ultimo_numero += "3"
                elif dedos_detectados == 4:
                    ultimo_numero += "4"
                elif dedos_detectados == 5:
                    ultimo_numero += "5"

            # Dibujar las conexiones de la mano
            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)

    # Si han pasado 5 segundos desde la última detección, reinicia la operación
    if time.time() - tiempo_ultima_deteccion > tiempo_reinicio:
        operacion_actual = ""
        resultado = ""
        ultimo_numero = ""
        mostrar_resultado = False  # Desbloquear entradas después del reinicio
    
    for i, imagen in enumerate(chuleta_imagenes):
        x_offset = 10 + i * 110  # Espacio entre imágenes
        # Coloca cada imagen en la parte superior de la ventana
        img[10:110, x_offset:x_offset + 100] = imagen

    # Mostrar el resultado en la pantalla
    cv2.putText(img, f"Operacion: {resultado}{ultimo_numero}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Mostrar imagen
    cv2.imshow("WebCam", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Salir con 'q'
        break

cam.release()
cv2.destroyAllWindows()


I0000 00:00:1736275494.318745 5007804 gl_context.cc:357] GL version: 2.1 (2.1 INTEL-18.7.4), renderer: Intel(R) Iris(TM) Plus Graphics 640
W0000 00:00:1736275494.419258 5159871 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1736275494.442139 5159871 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
